In [44]:
#kutuphanelerin yuklenmesi
!pip install beautifulsoup4
!pip install requests
from bs4 import BeautifulSoup
import requests # sitelere istek gondermek icin
import numpy as np
import pandas as pd

In [45]:
#sayfa linkinin verilmesi
link ="https://www.imdb.com/chart/top/?ref_=nv_mv_250"

#sayfa linkinin url olarak isimlendirilmesi
url= link

headers={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36"}

#sayfaya istek atılması
r = requests.get(url , headers = headers)

#sayfa kaynagını al.
soup = BeautifulSoup(r.content,"lxml")

#film_name ve film_rating adında ıkı bos liste olusturulması
film_name = []
film_rating = []

#film baslıklarının hepsini cekilmesini saglamaktadır. For dongusu ile de bu isimler tek tek cekilip film_name listesinin icine atılacaktır.
soup_title = soup.find_all("td", attrs={"class":"titleColumn"})  

for names in soup_title:
  
  try:
    #cekilen film ismi varsa film_name icine ismi yolla
    film_name.append(names.text.strip())
    
  except:
    #cekilen film ismi yoksa film_rating icine bos deger yolla
    film_name.append(np.NaN)
    

#film puanlarının hepsini cekilmesini saglamaktadır. For dongusu ile de bu puanlar tek tek cekilip film_rating listesinin icine atılacaktır.
soup_rating = soup.find_all("td", attrs={"class":"ratingColumn imdbRating"}) 

for rating in soup_rating:
  
  
  try:
    #cekilen puan varsa film_rating icine puanı yolla
    film_rating.append(rating.text.strip())
    
  except:
    #cekilen puan yoksa film_rating icine bos deger yolla
    film_rating.append(np.NaN)
    

In [46]:
#film_name'nin dataframe'ye cevrilmesi
film_name = pd.DataFrame(film_name )

#kolon ismi verilmesi
film_name.columns = ["film_name"]

#film_rating'in dataframe'ye cevrilmesi
film_rating = pd.DataFrame(film_rating ,columns = ["film_rating"])

#kolon ismi verilmesi
film_rating.columns = ["film_rating"]

#film_name ve film_rating'in birlestirilmesi
df_all = pd.concat([film_name  , film_rating] , axis = 1)

#ilk 2 gozlemin gosterilmesi
df_all.head(2)

,film_name,film_rating
0,1.\n The Shawshank Redemption\n(1994),9.2
1,2.\n The Godfather\n(1972),9.1


# Veinin Temizlenmesi


In [47]:
#verinin tiplerinin gosterilmesi
df_all.dtypes

film_name      object
film_rating    object
dtype: object

In [48]:
#Rating'in float degere cevirilmesi gerekmektedir.
df_all.film_rating = df_all.film_rating.astype("float")

In [49]:
#film_name 'in isimleri ve tarihleri olarak iki ayrılması. Ve film_name'nin alınması.
df_all["film_names"] = df_all["film_name"].apply(lambda x : x.split("\n")[1] )

In [50]:
#film_name 'in isimleri ve tarihleri olarak iki ayrılması. Ve film_date'in alınması.
df_all["film_date"] = df_all["film_name"].apply(lambda x : x.split("\n")[2] )

In [51]:
#ilk 3 gozlemın gosterilmesi
df_all.head(3)

,film_name,film_rating,film_names,film_date
0,1.\n The Shawshank Redemption\n(1994),9.2,The Shawshank Redemption,(1994)
1,2.\n The Godfather\n(1972),9.1,The Godfather,(1972)
2,3.\n The Godfather: Part II\n(1974),9.0,The Godfather: Part II,(1974)


In [52]:
#film_name' gerek kalmadığı için düşürüyoruz.

df_all = df_all.drop(["film_name"] , axis = 1)

In [53]:
#Tarih kısmının işlenebilmesi için parantezlerden kurtulmamız lazım.

df_all["film_date"] = df_all["film_date"].str.replace("(","")
df_all["film_date"] = df_all["film_date"].str.replace(")","")

In [54]:
#film_date object oldugu icin  sayısal degerlere cevirmemiz gerekir. Çünkü veri bilimi işlemlerinde kullanmamız gerekiyor.

df_all["film_date"] = df_all["film_date"].astype("int")

In [55]:
#verinin tiplerinin kontrol edilmesi
df_all.dtypes

film_rating    float64
film_names      object
film_date        int64
dtype: object

In [56]:
#ilk 3 gozlemin gosterilmesi
df_all.head(3)

,film_rating,film_names,film_date
0,9.2,The Shawshank Redemption,1994
1,9.1,The Godfather,1972
2,9.0,The Godfather: Part II,1974


# Verinin İndirilmesi

In [ ]:
#verinin csv olarak kaydedilmesi
df_all.to_csv(r"imdb_data.csv",encoding="utf-8", index=False , mode="a")

#df.to_csv(r"kitap_tavsiye_123.csv",encoding="utf-8", index=False , mode="a")
from google.colab import files
files.download('imdb_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>